In [29]:
from Bio import SeqIO
import pandas as pd

In [25]:
geneDB = []
with open("small.fasta","r") as geneF:
    geneDB = list(SeqIO.parse(geneF, "fasta"))

In [45]:
k = 3
kmers = {}
for seqrec in geneDB[0:5]:
    seq = seqrec.seq
    for i in range(0, len(seq)-k+1):
        ind = str(seq[i:i+k])
        try:
            kmers[ind] = kmers[ind] + [(seqrec.id, i)]
        except:
            kmers[ind] = [(seqrec.id, i)]
        

In [47]:
testseq = """MSIIGATRLQNDKSDTYSAGPCYAGGCSAFTPRGTCGKDWDLGEQTCASGFCTSQPLCAR
IKKTQVCGLRYSSKGKDPLVSAEWDSRGAPYVRCTYDADLIDTQAQVDQFVSMFGESPSL
AERYCMRGVKNTAGELVSRVSSDADPAGGWCRKWYSAHRGPDQDAALGSFCIKNPGAADC
KCINRASDPVYQKVKTLHAYPDQCWYVPCAADVGELKMGTQRDTPTNCPTQVCQIVFNML
DDGSVTMDDVKNTINCDFSKYVPPPPPPKPTPPTPPTPPTPPTPPTPPTPPTPRPVHNRK
VMFFVAGAVLVAILISTVRW"""

In [51]:
count = {}
for i in range(0, len(testseq)-k+1):
    ind = testseq[i:i+k]
    try:
        kmers[ind]
        try:
            count[ind] = (count[ind][0]+1, count[ind][1]+[i], count[ind][1])
        except:
            count[ind] = (1, [i], kmers[ind])
    except:
        pass
count

{'MSI': (1, [0], [('sp|Q6GZX3|002L_FRG3G', 0)]),
 'SII': (1, [1], [('sp|Q6GZX3|002L_FRG3G', 1)]),
 'IIG': (1, [2], [('sp|Q6GZX3|002L_FRG3G', 2)]),
 'IGA': (1, [3], [('sp|Q6GZX3|002L_FRG3G', 3), ('sp|Q6GZX2|003R_FRG3G', 357)]),
 'GAT': (1, [4], [('sp|Q6GZX3|002L_FRG3G', 4), ('sp|Q6GZX2|003R_FRG3G', 130)]),
 'ATR': (1, [5], [('sp|Q6GZX3|002L_FRG3G', 5)]),
 'TRL': (1, [6], [('sp|Q6GZX3|002L_FRG3G', 6), ('sp|Q197F8|002R_IIV3', 157)]),
 'RLQ': (1, [7], [('sp|Q6GZX3|002L_FRG3G', 7)]),
 'LQN': (1, [8], [('sp|Q6GZX3|002L_FRG3G', 8)]),
 'QND': (1, [9], [('sp|Q6GZX3|002L_FRG3G', 9)]),
 'NDK': (1, [10], [('sp|Q6GZX3|002L_FRG3G', 10)]),
 'DKS': (1, [11], [('sp|Q6GZX3|002L_FRG3G', 11)]),
 'KSD': (1, [12], [('sp|Q6GZX3|002L_FRG3G', 12)]),
 'SDT': (1, [13], [('sp|Q6GZX3|002L_FRG3G', 13)]),
 'DTY': (1,
  [14],
  [('sp|Q6GZX3|002L_FRG3G', 14), ('sp|Q197F7|003L_IIV3', 127)]),
 'TYS': (1, [15], [('sp|Q6GZX3|002L_FRG3G', 15)]),
 'YSA': (2, [16, 156], [16]),
 'SAG': (1, [17], [('sp|Q6GZX3|002L_FRG3G', 17)]